<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

This exercise covers two aspects:
* In tasks 1-6 you will implement mechanisms that allow training deeper models (better initialization, batch normalization). Note that for dropout and batch norm you are expected to implement it yourself without relying on ready-made components from Pytorch.
* In task 7 you will implement a convnet using [conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).


Tasks:
1. Check that the given implementation reaches 95% test accuracy for
   architecture input-64-64-10 in a few thousand batches.
2. Improve initialization and check that the network learns much faster
   and reaches over 97% test accuracy. A good basic initialization scheme is so-called Glorot initialization. For a set of weights going from a layer with $n_{in}$ neurons to a layer with $n_{out}$ neurons, it samples each weight from normal distribution with $0$ mean and standard deviation of $\sqrt{\frac{2}{n_{in}+n_{out}}}$.
3. Check, that with proper initialization we can train architecture
   input-64-64-64-64-64-10, while with bad initialization it does
   not even get off the ground.
4. Add dropout implemented in pytorch
5. Check that with 10 hidden layers (64 units each) even with proper
    initialization the network has a hard time to start learning.
6. Implement batch normalization (use train mode also for testing - it should perform well enough):
    * compute batch mean and variance
    * add new variables beta and gamma
    * check that the networks learns much faster for 5 layers
    * check that the network learns even for 10 hidden layers.
7. So far we worked with a fully connected network. Design and implement in pytorch (by using pytorch functions) a simple convolutional network and achieve 99% test accuracy. The architecture is up to you, but even a few convolutional layers should be enough.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
class Linear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        # self.weight.data.normal_(mean=0,std=0.25)
        self.weight.data.normal_(mean=0,std=np.sqrt(2/(self.in_features + self.out_features)))

        init.zeros_(self.bias)

    def forward(self, x):
        r = x.matmul(self.weight.t())
        r += self.bias
        return r

class Dropout(torch.nn.Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p

        if self.p < 0 or self.p > 1:
            raise ValueError("p must be between 0 and 1")
      
    def forward(self, x):
        if self.training:
            x.mul(torch.empty(x.size()[1]).uniform_(0, 1) >= self.p) * (1 / (1 - self.p))
        return x

class BatchNorm(torch.nn.Module):
    def __init__(self, num_features, eps=1e-05, momentum=0.1):
        super(BatchNorm, self).__init__()

        self.beta = Parameter(torch.Tensor(num_features))
        self.gamma = Parameter(torch.Tensor(num_features))

        self.eps = eps
        self.momentum = momentum

        self.reset_parameters()

    def reset_parameters(self):
        init.zeros_(self.beta)
        init.ones_(self.gamma)   

    def forward(self, x):
        batch_mean = torch.mean(x, dim=0)
        batch_var = torch.var(x, dim=0, unbiased=True)

        x_norm = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        x_rescaled = x_norm * self.gamma + self.beta

        return x_rescaled



class Net(nn.Module):
    def __init__(self, hidden_layers_sizes=[64, 64, 64]):
        super(Net, self).__init__()

        weights_shapes = zip([784]+hidden_layers_sizes, hidden_layers_sizes+[10])
        self.linears = nn.ModuleList(Linear(n_in, n_out) for n_in, n_out in weights_shapes)

    def forward(self, x):
        x = x.view(-1, 28 * 28)

        for layer_index, fc in enumerate(self.linears):
          x = fc(x)
          if layer_index < len(self.linears)-1:
            x = BatchNorm(fc.out_features)(x)
            x = Dropout(0.5)(x)
            x = F.relu(x)

        output = F.log_softmax(x, dim=1)
        return output


In [3]:
class MnistTrainer(object):
    def __init__(self, batch_size, net):
        self.net = net
        transform = transforms.Compose(
                [transforms.ToTensor()])
        self.trainset = torchvision.datasets.MNIST(
            root='./data',
            download=True,
            train=True,
            transform=transform)
        self.trainloader = torch.utils.data.DataLoader(
            self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

        self.testset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(
            self.testset, batch_size=batch_size, shuffle=False, num_workers=2)

    def train(self):
        net = self.net

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

        for epoch in range(20):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} test images: {} %'.format(
                total, 100 * correct / total))

In [ ]:
hidden_layers_sizes = [64] * 10
net = Net(hidden_layers_sizes=hidden_layers_sizes)
trainer = MnistTrainer(batch_size=128, net=net)
trainer.train()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

[1,   100] loss: 0.782
[1,   200] loss: 0.328
[1,   300] loss: 0.255
[1,   400] loss: 0.220
Accuracy of the network on the 10000 test images: 93.98 %
[2,   100] loss: 0.167
[2,   200] loss: 0.164
[2,   300] loss: 0.159
[2,   400] loss: 0.156
Accuracy of the network on the 10000 test images: 95.46 %
[3,   100] loss: 0.113
[3,   200] loss: 0.126
[3,   300] loss: 0.122
[3,   400] loss: 0.114
Accuracy of the network on the 10000 test images: 96.03 %
[4,   100] loss: 0.096
[4,   200] loss: 0.094
[4,   300] loss: 0.107
[4,   400] loss: 0.095
Accuracy of the network on the 10000 test images: 95.89 %
[5,   100] loss: 0.083
[5,   200] loss: 0.079
[5,   300] loss: 0.086
[5,   400] loss: 0.097
Accuracy of the network on the 10000 test images: 96.3 %
[6,   100] loss: 0.074
[6,   200] loss: 0.076
[6,   300] loss: 0.071
[6,   400] loss: 0.083
Accuracy of the network on the 10000 test images: 96.47 %
[7,   100] loss: 0.057
[7,